In [2]:
from pymilvus import connections, Collection
from sentence_transformers import SentenceTransformer

# --- 1. 配置 (保持和之前一致) ---
# 请确保这里的 IP 和您之前运行成功时的 IP 一致
MILVUS_HOST = "192.168.16.138" 
MILVUS_PORT = "19530"
COLLECTION_NAME = "report_analysis"
MODEL_NAME = 'all-mpnet-base-v2'

In [3]:
def main():
    # --- 连接数据库 ---
    print(f"正在连接 Milvus ({MILVUS_HOST})...")
    try:
        connections.connect("default", host=MILVUS_HOST, port=MILVUS_PORT)
        collection = Collection(COLLECTION_NAME)
        collection.load() # 搜索前必须加载到内存
        print(f"连接成功！当前集合内共有 {collection.num_entities} 条数据。")
    except Exception as e:
        print(f"连接失败: {e}")
        return

    # --- 任务一：肉眼检查 (查看前 5 条数据) ---
    print("\n" + "="*20 + " 任务一：数据内容检查 " + "="*20)
    print("正在从数据库中取出前 5 条数据，请检查文字是否完整...\n")
    
    # query 接口类似 SQL 的 select，用于获取具体字段
    results = collection.query(
        expr="chunk_id >= 0",  # 条件：获取所有ID
        output_fields=["chunk_id", "text_content"], # 我们只想看内容
        limit=5 # 只看前5条
    )

    for res in results:
        content = res['text_content']
        cid = res['chunk_id']
        print(f"--- [Chunk ID: {cid}] (长度: {len(content)}) ---")
        # 打印前100个字和最后50个字，检查头尾
        print(f"开头: {content[:80]}...") 
        print(f"结尾: ...{content[-50:]}")
        print("-" * 50)

    # --- 任务二：语义搜索测试 (模拟真实使用) ---
    print("\n" + "="*20 + " 任务二：语义检索测试 " + "="*20)
    question = "资本对文娱产业有什么影响？"
    print(f"正在测试提问：'{question}'")
    
    # 1. 把问题变成向量
    print("正在加载模型生成向量...")
    model = SentenceTransformer(MODEL_NAME)
    query_vector = model.encode([question])
    
    # 2. 在 Milvus 里搜索最相似的片段
    search_params = {"metric_type": "L2", "params": {"nlist": 128}}
    results = collection.search(
        data=query_vector, 
        anns_field="vector", 
        param=search_params, 
        limit=2, # 找最相似的2个
        output_fields=["text_content"] # 同时返回文本内容
    )

    # 3. 打印结果
    print(f"\n找到 {len(results[0])} 个相关片段：")
    for i, hit in enumerate(results[0]):
        score = hit.distance # 距离越小越相似 (L2距离)
        text = hit.entity.get("text_content")
        print(f"\n>>> 结果 {i+1} (相似度距离: {score:.4f}):")
        print(text)

if __name__ == "__main__":
    main()

正在连接 Milvus (192.168.16.138)...
连接成功！当前集合内共有 13 条数据。

==================== 任务一：数据内容检查 ====================
正在从数据库中取出前 5 条数据，请检查文字是否完整...

--- [Chunk ID: 462116480351735625] (长度: 469) ---
开头: 〖特别报告〗      政府左右房价？（下）2021年06月02日福卡分析                                  总字数：5694〖...
结尾: ...国未来最具发展潜力、市场最活跃的行业之一。而随着文娱产业潮起，自然引得竞争者们铆足了劲“乘风破浪”。
--------------------------------------------------
--- [Chunk ID: 462116480351735626] (长度: 495) ---
开头: 2020年全国规模以上文化及相关产业企业实现营业收入98514亿元，其中，互联网+其他信息服务、其他文化数字内容服务、可穿戴智能文化设备制造等5个行业小类营业收...
结尾: ...成为了一场裹挟着各路资本套路的“买卖”，如近期“倒奶事件”便是在“唯钱是举”的打投应援机制下的产物。
--------------------------------------------------
--- [Chunk ID: 462116480351735627] (长度: 447) ---
开头: “你我本无缘，全靠我花钱”也直接催生了诸多追星金融套路，最具争议的莫过于集资，据蓝鲸财经报道，截至4月5日，《创造营2021》与《青春有你3》已经有超5位选手后...
结尾: ...僭越其工具属性成为人们全部生活的价值旨归。究其原因，既因流量至上“博眼球”。互联网时代，流量即资源。
--------------------------------------------------
--- [Chunk ID: 462116480351735628] (长度: 383) ---
开头: 而随着互联网用户红利的日渐消弭，在“流量=用户数×用户时长”的规则中，尽可能多地占领用户时长成为一种共识，“博眼球”之风渐成趋势。而用户在持续